In [27]:
import pyspark
from pyspark.sql import SparkSession, functions 
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.read.text("NASA_access_log_Aug95")

In [4]:
df1 = spark.read.text("NASA_access_log_Jul95")

In [7]:
df_concat = df.union(df1)

In [9]:
df_concat.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


In [9]:
df2 = df_concat.select(
            split(col("value"),"- -")[0].alias("hostname")
       )

## 1. Número de hosts únicos.

In [10]:
distinctHosts = df2.select(countDistinct("hostname").alias("uniq_hosts"))

In [11]:
distinctHosts.show()

+----------+
|uniq_hosts|
+----------+
|    137979|
+----------+



## 2. O total de erros 404.

In [12]:
df3 = df_concat.withColumn("cod_404", regexp_extract(df_concat.value, r"( 404 )", 0) )

In [14]:
df_cod_404 = df3.where(col("cod_404") == lit(" 404 "))

In [15]:
df_cod_404.count()

20901

## 3. Os 5 URLs que mais causaram erro 404.

In [16]:
df4 = df_cod_404.select(
            split(col("value"),"- -")[0].alias("hostname")
       )

In [17]:
df4.groupBy("hostname").count().orderBy(col("count"),ascending=False).limit(5).show(truncate=False)

+----------------------------+-----+
|hostname                    |count|
+----------------------------+-----+
|hoohoo.ncsa.uiuc.edu        |251  |
|piweba3y.prodigy.com        |157  |
|jbiagioni.npt.nuwc.navy.mil |132  |
|piweba1y.prodigy.com        |114  |
|www-d4.proxy.aol.com        |91   |
+----------------------------+-----+



## 4. Quantidade de erros 404 por dia.

In [18]:
df5 = df_cod_404.select(
            split(col("value"),"]")[0].alias("aux_date")
       )

In [19]:
df6 = df5.select(
            split(col("aux_date"),"- -")[1].alias("date2")
       )

In [20]:
df6 = df6.withColumn('date', regexp_replace(df6.date2, '\[|:.*', '')).drop("date2")

In [150]:
df6.groupBy("date").count().orderBy(col("date")).show(10,False)

+------------+-----+
|date        |count|
+------------+-----+
| 01/Aug/1995|243  |
| 01/Jul/1995|316  |
| 02/Jul/1995|291  |
| 03/Aug/1995|304  |
| 03/Jul/1995|474  |
| 04/Aug/1995|346  |
| 04/Jul/1995|359  |
| 05/Aug/1995|236  |
| 05/Jul/1995|497  |
| 06/Aug/1995|373  |
+------------+-----+
only showing top 10 rows



## 5. O total de bytes retornados.

In [22]:
df7 = df3.filter(col("cod_404") != '404')

In [23]:
df8 = df7.select(
            split(col("value"),'" ')[1].alias("aux_total_bytes")
       )

In [24]:
df9 = df8.select(
            split(col("aux_total_bytes")," ")[1].alias("aux_total_bytes2")
       )

In [25]:
df10 = df9.withColumn('total_bytes2', regexp_replace(df9.aux_total_bytes2, '-', '0')).drop("aux_total_bytes2")

In [26]:
df11 = df10.withColumn("total_bytes", df10["total_bytes2"].cast(IntegerType())).drop("total_bytes2")

## Primeira forma de realizar o group by

In [208]:
total_bytes = df11.groupBy().sum()

In [205]:
total_bytes.show()

+----------------+
|sum(total_bytes)|
+----------------+
|     65524307881|
+----------------+



## Segunda forma de realizar o group by

In [206]:
df11.agg(sum("total_bytes").alias("total_bytes")).show()

+-----------+
|total_bytes|
+-----------+
|65524307881|
+-----------+

